In [ ]:
import import_ipynb
import os
import shutil
import glob
import csv
import cv2
import scipy

from seg_functions_TC import*

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
from PIL import Image, ImageDraw, ImageEnhance
from scipy.spatial import ConvexHull
from skimage import measure

In [ ]:
def filtered_images(ct_image_2d):
    
    HUbone = 200 #bone is around 300-400 HU (taking artefacts such as streaking- and cupping artefacts in account so decreasing to 200)
    ret,thresh = cv2.threshold(ct_image_2d,HUbone,3000,cv2.THRESH_BINARY)
    
    # om te voorkomen dat je een of andere vage error over orde van grootte van de getallen krijgt
    thres = thresh.astype('int16')
    bone_threshold_filtered_image = cv2.medianBlur(thres, 3)
    
    return bone_threshold_filtered_image

In [ ]:
def find_most_anterior_lung_points(lung_mask):
    # Check if the image is correctly oriented e.g. the sternum at the top, vertebrea at the bottom of the image,
    
    lung_mask_90 = np.rot90(lung_mask)
    arr = np.nonzero(lung_mask_90)
    
    l1 = [arr[0][0]],[arr[1][0]]
    
    # Marker labelling to to able to distinct between masks
    ret, markers = cv2.connectedComponents(lung_mask_90)

    # één masker verwijderen
    markers[markers == markers[l1]] = 0 # markers in markers die ongelijk zijn aan de waarde van het eerste punt 
    arr2 = np.nonzero(markers)
    l2 = [arr2[0][0]],[arr2[1][0]] 
    # [arr2[as (0=rij of 1=kolom)][hoeveelste indices]],[arr2[as (0=rij of 1=kolom)][hoeveelste indices]] !!!

#     imgplot = plt.imshow(lung_mask_90, cmap='gray')
#     plt.plot(l1[1], l1[0], marker='o', markersize=5, color="red")
#     plt.plot(l2[1], l2[0], marker='o', markersize=5, color="blue")
    
    # for later implementation:
    # if not fill in 1: for 90 degree rotation, 2: for 180 or 3: 270 degree rotation.
    
    return lung_mask_90, l1, l2

In [ ]:
def centroid_sternum(lung_mask_90, bone_threshold_filtered_image, l1, l2):


    # set values of columns before the first point and points after column of the second point to zero (of rotated original image)
    fov_between_lungs = l1[1], l2[1]
    fov_between_lungs = np.sort(fov_between_lungs, axis=None)
    rows = np.prod(lung_mask_90[1].shape)
    bone_threshold_filtered_image_90 = np.rot90(bone_threshold_filtered_image)
    mask = np.zeros(lung_mask_90.shape)
    mask[l1[0][0]-50:l1[0][0]+50,fov_between_lungs[0]:fov_between_lungs[1]]=1

    middle_col = np.around((fov_between_lungs[0]+fov_between_lungs[1])/2)
    # middle_row = np.around()
    print(fov_between_lungs[0],fov_between_lungs[1])
    fov_im90 = bone_threshold_filtered_image_90*mask
#     imgplot = plt.imshow(fov_im90, cmap='gray')
    # plt.plot(middle_point, 140, marker='o', markersize=5, color="blue") # middle of the two lung points

    # posterior middelpunt sternum bepalen:

    #closing uitvoeren
    kernel = np.ones((5,5),np.uint8)
    closing = cv2.morphologyEx(fov_im90, cv2.MORPH_CLOSE, kernel, iterations=3)
    closing = closing.astype('int8')# om te voorkomen dat je een of andere vage error over orde van grootte van de getallen krijgt
    imgplot = plt.imshow(closing, cmap='gray')


    # Choose 4 or 8 for connectivity type
    connectivity = 4  
    masks = cv2.connectedComponentsWithStats(closing, connectivity, cv2.CV_32S)

    num_labels = masks[0]
    stats = masks[2]
    centroids = masks[3]

    for label in range(1,num_labels):
        blob_area = stats[label, cv2.CC_STAT_AREA]
        blob_width = stats[label, cv2.CC_STAT_WIDTH]
        blob_height = stats[label, cv2.CC_STAT_HEIGHT]

    mask_area = stats[:,4]
    filtered_mask_area = np.sort([x for x in mask_area if x <= 100000 and x > 50])
    a = stats[:,4] == filtered_mask_area[0]
    b=[i for i, x in enumerate(a) if x]
    centroid_sternum = np.around(centroids[b])
    
    
    row_centroid_sternum = np.uint(centroid_sternum[0][1])
    col_centroid_sternum = np.uint(centroid_sternum[0][0])
    centroid_axis = closing[:,col_centroid_sternum]
    centroid_axis_corr = centroid_axis[row_centroid_sternum:]

    # find indices where the value changes.
    index_backgroud_mask = np.where(centroid_axis_corr[:-1] != centroid_axis_corr[1:])[0] 
    # add the value of the first index to the centroid row value
    posterior_centroid_sternum_row = index_backgroud_mask[0]+row_centroid_sternum
    
    return posterior_centroid_sternum_row, col_centroid_sternum